Sentiment Analysis using LLM (Meta Llama 3.2-1B-Instruct )

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import json
import re
import os
import time

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
from huggingface_hub import login
login("hf_your_token_here")  # Replace with your actual token

In [ ]:
def load_finance_llama(model_id="meta-llama/Llama-3.2-1B-Instruct"):
    print("Loading LLaMA model...")
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True

        )
    except Exception as e:
        print("GPU load failed, using CPU fallback...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="cpu",
            low_cpu_mem_usage=True

        )

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer

    )
    print("Meta LLaMA LLM loaded successfully.")
    return generator

In [ ]:
#If the news mentions “Jio”, “Reliance Jio”, “Jio Financial”, or “Jio Platforms”, treat them as part of Reliance Industries.
def sentiment_prompt(company, text):
    return f"""
You are a precise **financial sentiment analyst**.
Your goal is to determine how the following news article about **{company}** would affect market sentiment and short-term stock movement for the company {company}.
For the following financial news article, do the following:

  sentiment: Determine the degree of sentiment of the news article with respect to {company}.
    Output in the range of (-1, 1) where -1 is Negative, and 1 is Positive.
  stock_movement: Given your expertise in the field, determine stock movement of the {company}.
    Output as Up or Down.
  reason : 1-2 sentence explanation referencing specific metrics, events, or facts from the news.

Do NOT explain your reasoning or include any text outside the JSON.

### Output format
{{
  "sentiment_score": float (-1.0 to 1.0),
  "predicted_direction": "Up" | "Neutral" | "Down",
  "reason": "1-2 sentence explanation referencing specific metrics, events, or facts from the news"
}}

Be concise, confident and consistent. DO NOT include ANY commentary or repeated text.

Now analyze this:
"{text}"
"""


In [ ]:
def analyze_batch(texts, company, generator, batch_size=4):
    results = []
    num_batches = int(np.ceil(len(texts) / batch_size))
    start_time = time.time()
    batch_times = []

    for i in tqdm(range(0, len(texts), batch_size), desc="LLaMA Sentiment",ncols=90):
        batch_start = time.time()
        batch = texts[i:i + batch_size]
        batch_prompts = [sentiment_prompt(company, t) for t in batch]

        try:
            outputs = generator(
                batch_prompts,
                do_sample=True,
                temperature=0.3,
                top_p=0.9,
                max_new_tokens=128,
                pad_token_id=generator.tokenizer.eos_token_id
            )

            if isinstance(outputs, list) and all(isinstance(o, list) for o in outputs):
                outputs = [o[0] for o in outputs if len(o) > 0]

            for output in outputs:
              text_out = output.get("generated_text", "")
              json_match = re.search(r"\{.*\}", text_out, re.DOTALL)
              if json_match:
                  try:
                      parsed = json.loads(json_match.group(0))
                  except json.JSONDecodeError:
                      parsed = {"sentiment_score": None, "sentiment_label": None, "predicted_direction": None, "reason": text_out.strip()}
              else:
                  parsed = {"sentiment_score": None, "sentiment_label": None, "predicted_direction": None, "reason": text_out.strip()}

              results.append(parsed)

        except Exception as e:
            print(f"\n⚠️ Batch error at batch {i//batch_size}: {e}")
            for _ in batch:
                results.append({"sentiment_score": None, "sentiment_label": None, "predicted_direction": None, "reason": "Batch failed."})

        batch_time = time.time() - batch_start
        batch_times.append(batch_time)
        avg_time = np.mean(batch_times)
        remaining_batches = num_batches - (i // batch_size + 1)
        eta = remaining_batches * avg_time / 60

        print(f"Batch {i // batch_size + 1}/{num_batches} done in {batch_time:.1f}s | ETA ≈ {eta:.1f} min")

    total_time = (time.time() - start_time) / 60
    print(f"\nAll {num_batches} batches done in {total_time:.1f} min (avg {np.mean(batch_times):.1f}s per batch).")

    return results

In [ ]:
def llm_sentiment_pipeline(file_path, company_name, generator, batch_size=4, autosave_interval=2):
    print(f"Processing file: {file_path}")
    save_path = file_path.replace(".csv", "_with_llm_sentiments.csv")
    df = pd.read_csv(file_path)
    if "combined_text" not in df.columns:
        raise ValueError("'combined_text' column missing from file.")


    if os.path.exists(save_path):
        df_saved = pd.read_csv(save_path)
        start_index = len(df_saved)
        print(f"Resuming from batch {start_index // batch_size} ({start_index} rows done).")
    else:
        df_saved = pd.DataFrame()
        start_index = 0

    total = len(df)
    all_results = []

    for i in range(start_index, total, batch_size):
        batch_texts = df["combined_text"].iloc[i:i + batch_size].tolist()
        try:
            results = analyze_batch(batch_texts, company_name, generator, batch_size=batch_size)
        except Exception as e:
            print(f"⚠️ Error at batch {i // batch_size}: {e}")
            continue

        df_llm = pd.DataFrame(results)
        df_batch = pd.concat([df.iloc[i:i + batch_size].reset_index(drop=True), df_llm], axis=1)
        all_results.append(df_batch)


        if (i // batch_size + 1) % autosave_interval == 0 or i + batch_size >= total:
            combined_df = pd.concat([df_saved] + all_results, ignore_index=True)
            combined_df.to_csv(save_path, index=False)
            df_saved = combined_df.copy()
            all_results = []
            print(f"Saved progress at {i + batch_size}/{total} rows ({round((i + batch_size) / total * 100, 2)}%)")

        time.sleep(0.5)

    print(f"Finished processing. Saved final file → {save_path}")
    return df_saved

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
infosys_file = "/content/drive/MyDrive/processed/clean_infosys_with_sentiments.csv"
reliance_file = "/content/drive/MyDrive/processed/clean_reliance_with_sentiments.csv"
sbi_file = "/content/drive/MyDrive/processed/clean_sbi_with_sentiments.csv"
sbi_ind_file = "/content/drive/MyDrive/processed/clean_sbi_industry_with_sentiments.csv"

In [ ]:
generator = load_finance_llama()

Loading LLaMA model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


Meta LLaMA LLM loaded successfully.


In [ ]:
#Infosys News
company = "Infosys"
df_infosys = llm_sentiment_pipeline(infosys_file, company, generator, batch_size=4, autosave_interval=1)

Processing file: /content/drive/MyDrive/processed/clean_infosys_with_sentiments.csv


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:05<00:00,  5.55s/it]

Batch 1/1 done in 5.6s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 5.6s per batch).
Saved progress at 4/303 rows (1.32%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:06<00:00,  6.19s/it]

Batch 1/1 done in 6.2s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 6.2s per batch).
Saved progress at 8/303 rows (2.64%)


...


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:08<00:00,  8.65s/it]

Batch 1/1 done in 8.6s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 8.6s per batch).
Saved progress at 300/303 rows (99.01%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it]

Batch 1/1 done in 4.9s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 4.9s per batch).
Saved progress at 304/303 rows (100.33%)


Finished processing. Saved final file → /content/drive/MyDrive/processed/clean_infosys_with_sentiments_with_llm_sentiments.csv


In [ ]:
#SBI News
company = "SBI"
df_reliance = llm_sentiment_pipeline(sbi_file, company, generator, batch_size=6, autosave_interval=1)

Processing file: /content/drive/MyDrive/processed/clean_sbi_with_sentiments.csv


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:10<00:00, 10.50s/it]

Batch 1/1 done in 10.5s | ETA ≈ 0.0 min

All 1 batches done in 0.2 min (avg 10.5s per batch).
Saved progress at 6/526 rows (1.14%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:07<00:00,  7.36s/it]

Batch 1/1 done in 7.4s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 7.4s per batch).
Saved progress at 12/526 rows (2.28%)


...


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:10<00:00, 10.02s/it]

Batch 1/1 done in 10.0s | ETA ≈ 0.0 min

All 1 batches done in 0.2 min (avg 10.0s per batch).
Saved progress at 522/526 rows (99.24%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:06<00:00,  6.61s/it]

Batch 1/1 done in 6.6s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 6.6s per batch).
Saved progress at 528/526 rows (100.38%)


Finished processing. Saved final file → /content/drive/MyDrive/processed/clean_sbi_with_sentiments_with_llm_sentiments.csv


In [ ]:
#Reliance News
company = "Reliance Industries"
df_reliance = llm_sentiment_pipeline(reliance_file, company, generator, batch_size=4, autosave_interval=1)

Processing file: /content/drive/MyDrive/processed/clean_reliance_with_sentiments.csv


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:08<00:00,  8.72s/it]

Batch 1/1 done in 8.7s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 8.7s per batch).
Saved progress at 4/203 rows (1.97%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:07<00:00,  7.41s/it]

Batch 1/1 done in 7.4s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 7.4s per batch).
Saved progress at 8/203 rows (3.94%)


...


LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:07<00:00,  7.04s/it]

Batch 1/1 done in 7.0s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 7.0s per batch).
Saved progress at 200/203 rows (98.52%)



LLaMA Sentiment: 100%|██████████████████████████████████████| 1/1 [00:07<00:00,  7.47s/it]

Batch 1/1 done in 7.5s | ETA ≈ 0.0 min

All 1 batches done in 0.1 min (avg 7.5s per batch).
Saved progress at 204/203 rows (100.49%)


Finished processing. Saved final file → /content/drive/MyDrive/processed/clean_reliance_with_sentiments_with_llm_sentiments.csv
